### Test the API endpoint
After deploying the model, we'll use the specified endpoint and give the model a try.

Since we trained a model using Boston housing dataset, we'll use the same dataset to test the API.

In [3]:
from sklearn.datasets import load_boston

boston_data = load_boston()
# try 1 row of data to send to API
example_data = boston_data['data'][0]
example_data

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [4]:
example_payload = example_data.tolist()
example_payload

[0.00632,
 18.0,
 2.31,
 0.0,
 0.538,
 6.575,
 65.2,
 4.09,
 1.0,
 296.0,
 15.3,
 396.9,
 4.98]

### Use the prepared code to call the API
Azure provides handy code for you to call the API.

The only thing you have to take note of is line 14, where you have to specify the request data.

In [11]:
# code provided in Azure
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# !!!
# Request data goes here
# should be a list of lists (list elements contains features of 1 data point)
data = {"data": [example_payload]
}

body = str.encode(json.dumps(data))

url = 'http://6a636f99-7ce6-41d3-9f15-3d4e26db5455.southeastasia.azurecontainer.io/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'[[29.966776672474822]]'
